# 2024 + 2025 MRSA Predictions with best model 

This is to isolate the actual values from the best-performing 2024 model, as well as extrapolate the model to 2025.

# Import Packages

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Import Data

In [2]:
county_year_df = pd.read_csv(rf'..\data\mrsa_data\clean_data\county-year_county_mrsa.csv')
county_year_df = county_year_df.sort_values(by=['County', 'Year'])
county_year_df = county_year_df.drop(
    columns=county_year_df.filter(like="infections_predicted").columns
)

df = county_year_df.copy()
sdoh_df = pd.read_csv(rf'..\data\sdoh_data\cleaned_attributes\combined_measure_data.csv')


hospital_types = [
    "Critical Access",
    "Free-Standing Rehabilitation Hospital",
    "Long-Term Acute Care",
    "Major Teaching",
    "Pediatric",
    "Rehabilitation Unit",
    "cl",
    "cm",
    "cs"
]

# infection & patient-day column names
infection_cols = {ht: f"infections_reported__{ht}" for ht in hospital_types}
patient_cols   = {ht: f"patient_days__{ht}"        for ht in hospital_types}
patient_day_cols = [c for c in df.columns if c.startswith("patient_days__")]
target_total_col = "Infections_Reported_Total"

# SDOH columns 
sdoh_cols = [
    c for c in sdoh_df.columns
    if c not in ["FIPS", "Year", "State", "County"]
]


# Create Data Tables

In [ ]:
#  original df 
df_base = df.copy()

# Merge SDOH 
df_sdoh = df.merge(sdoh_df, on=["Year", "State", "County"], how="left")

#  lag features 
df_sdoh_lag = df_sdoh.sort_values(["State", "County", "Year"]).copy()

for ht, inf_col in infection_cols.items():
    lag_col = f"{inf_col}_lag1"
    df_sdoh_lag[lag_col] = (
        df_sdoh_lag
        .groupby(["State", "County"])[inf_col]
        .shift(1)
    )

# drop rows where lag is missing (first year per county)
df_sdoh_lag = df_sdoh_lag.dropna(
    subset=[f"{infection_cols[ht]}_lag1" for ht in hospital_types]
)

In [20]:
# df_sdoh_lag.columns

# Tune model 
gives same params as `clean_modeleing_notebook.ipynb`

In [ ]:


lag_cols = [f"{infection_cols[ht]}_lag1" for ht in hospital_types]
feature_cols_rf_lag = (
    ["Year", "State", "County"] +
    patient_day_cols +
    sdoh_cols +
    lag_cols
)

def make_preprocessor(feature_cols, cat_cols=("State", "County")):
    """Build a ColumnTransformer that one-hot encodes categorical columns and passes through the rest."""
    cat_cols = [c for c in cat_cols if c in feature_cols]
    transformer = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
        ],
        remainder="passthrough"
    )
    return transformer


#  train years for tuning
df_tune = df_sdoh_lag[(df_sdoh_lag["Year"] >= 2017) & (df_sdoh_lag["Year"] <= 2023)].copy()

X_tune = df_tune[feature_cols_rf_lag]
y_tune = df_tune["Infections_Reported_Total"]

# build and fit preprocessor
preprocessor_tune = make_preprocessor(feature_cols_rf_lag)
X_tune_proc = preprocessor_tune.fit_transform(X_tune)

# RF + hyperparameter search space
base_rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": [200, 400, 600, 800],
    "max_depth": [None, 5, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.3, 0.5, 0.8],
    "bootstrap": [True, False],
}

rf_random = RandomizedSearchCV(
    estimator=base_rf,
    param_distributions=param_dist,
    n_iter=40,                     
    scoring="neg_mean_absolute_error",
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# rune tuning

rf_random.fit(X_tune_proc, y_tune)

print("Best params for RF_with_SDOH_Lag:")
print(rf_random.best_params_)

best_params_rf_lag = rf_random.best_params_




Fitting 3 folds for each of 40 candidates, totalling 120 fits
Best params for RF_with_SDOH_Lag:
{'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 0.8, 'max_depth': 10, 'bootstrap': True}


# 2024

In [23]:
target_total_col = "Infections_Reported_Total"

# build tuned rf
def build_rf_tuned():
    return RandomForestRegressor(
        n_estimators=best_params_rf_lag.get("n_estimators", 500),
        max_depth=best_params_rf_lag.get("max_depth", None),
        min_samples_split=best_params_rf_lag.get("min_samples_split", 2),
        min_samples_leaf=best_params_rf_lag.get("min_samples_leaf", 1),
        max_features=best_params_rf_lag.get("max_features", "sqrt"),
        bootstrap=best_params_rf_lag.get("bootstrap", True),
        random_state=42,
        n_jobs=-1
    )



#2023 train, 2024 test
df_train = df_sdoh_lag[df_sdoh_lag["Year"] <= 2023].dropna(
    subset=[target_total_col]
).copy()

df_test = df_sdoh_lag[df_sdoh_lag["Year"] == 2024].dropna(
    subset=[target_total_col]
).copy()

X_train = df_train[feature_cols_rf_lag]
y_train = df_train[target_total_col]

X_test = df_test[feature_cols_rf_lag]
y_test = df_test[target_total_col]

preprocessor_final = make_preprocessor(feature_cols_rf_lag)

X_train_proc = preprocessor_final.fit_transform(X_train)
X_test_proc  = preprocessor_final.transform(X_test)

# fit model to params
rf_tuned = build_rf_tuned()
rf_tuned.fit(X_train_proc, y_train)

y_pred_2024 = rf_tuned.predict(X_test_proc)

# put in df format
pred_2024 = df_test[["State", "County", "Year"]].copy()
pred_2024["infections_predicted_rf_sdoh_lag"] = y_pred_2024
pred_2024["infections_actual"] = y_test.values

pred_2024

pred_2024.to_csv("pred_2024_rf_sdoh_lag.csv", index=False)
print("Saved pred_2024 to CSV.")


Saved pred_2024 to CSV.


# 2025

Since we don't have data for 2025, we will use 2024 data as both lag features, as well as a "proxy" for what 2024 will report.

How the 2025 Prediction Input Data Is Constructed:

- We start from all county-year rows for 2024, because no 2025 covariate data exists.
- We duplicate the 2024 rows and set `Year = 2025`, creating placeholder 2025 feature rows.
- These duplicated rows provide the baseline 2025 values for all non-lag features (SDOH, patient-days, demographics, etc.).
- Using the trained model, we previously generated 2024 infection predictions for each county.
- We merge these 2024 predicted infections into the duplicated 2025 rows.
- For every lag feature (`*_lag1`), we replace the 2025 lag value with the 2024 predicted infection for that county (the correct "previous year" value).
- All other features remain identical to 2024, since true 2025 covariate values are unavailable.
- The final constructed 2025 dataset (with updated lag features) is then passed through the preprocessor and Random Forest model to generate 2025 infection predictions.


In [24]:
# same as abovee to biuld model
def build_rf_tuned():
    return RandomForestRegressor(
        n_estimators=best_params_rf_lag.get("n_estimators", 500),
        max_depth=best_params_rf_lag.get("max_depth", None),
        min_samples_split=best_params_rf_lag.get("min_samples_split", 2),
        min_samples_leaf=best_params_rf_lag.get("min_samples_leaf", 1),
        max_features=best_params_rf_lag.get("max_features", "sqrt"),
        bootstrap=best_params_rf_lag.get("bootstrap", True),
        random_state=42,
        n_jobs=-1
    )


# 2024 up to  training
df_train = df_sdoh_lag[df_sdoh_lag["Year"] <= 2024].copy()

X_train = df_train[feature_cols_rf_lag]
y_train = df_train["Infections_Reported_Total"]

preprocessor_final = make_preprocessor(feature_cols_rf_lag)
X_train_proc = preprocessor_final.fit_transform(X_train)

rf_tuned = build_rf_tuned()
rf_tuned.fit(X_train_proc, y_train)

print("Training complete through 2024.")

# use 2025 as test
df_2024 = df_sdoh_lag[df_sdoh_lag["Year"] == 2024].copy()
df_2025 = df_2024.copy()
df_2025["Year"] = 2025   # forward-fill all non-lag features

# use 2024 values as a proxy for 2025 values for non-lag features

X_2024 = df_2024[feature_cols_rf_lag]
X_2024_proc = preprocessor_final.transform(X_2024)
df_2024["predicted_2024"] = rf_tuned.predict(X_2024_proc)


df_2025 = df_2025.merge(
    df_2024[["State", "County", "predicted_2024"]],
    on=["State", "County"],
    how="left"
)

# update  each lag column using predicted_2024
for ht in hospital_types:
    lag_col = f"infections_reported__{ht}_lag1"
    df_2025[lag_col] = df_2025["predicted_2024"]

# predict infections
X_2025 = df_2025[feature_cols_rf_lag]
X_2025_proc = preprocessor_final.transform(X_2025)

df_2025["infections_predicted_rf_sdoh_lag"] = rf_tuned.predict(X_2025_proc)

# Final output table
pred_2025 = df_2025[["State", "County", "Year", "infections_predicted_rf_sdoh_lag"]]

pred_2025.to_csv("pred_2025_rf_sdoh_lag.csv", index=False)
print("Saved pred_2025 to CSV.")


Training complete through 2024.
Saved pred_2025 to CSV.
